### Exercise 2b)

In [ ]:
from neural_network import NeuralNetwork
import numpy as np
from training import GradientDescent, StochasticGradientDescent
from step_methods import ConstantLearningRateStep, MomentumStep, ADAgradStep, RMSpropStep, AdamStep
from utils.cost_functions import SoftmaxCrossEntropy,MSE
from utils.activation_functions import Sigmoid, Identity
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

In [ ]:
def runge(x): 
    return 1/(1 + 25*x**2)

n = 500
x = np.random.uniform(-1, 1, n)
y = runge(x) + 0.05*np.random.normal(0, 1, n)

model = LinearRegression(fit_intercept=False)
model = make_pipeline(PolynomialFeatures(degree=12, include_bias=False), StandardScaler(), model)

x_train, x_test, y_train, y_test = train_test_split(x.reshape(-1,1), y.reshape(-1,1), test_size=0.2,random_state = 44) 
y_mean = y_train.mean()
y_pred = model.fit(x_train, y_train - y_mean).predict(x_test)
print("Testing MSE for OLS, with linear regression: ",np.mean((y_pred-(y_test - y_mean)))**2)

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

network_input_size = 1
network_output_size = 1

layer_output_sizes = [50, 50, network_output_size]
activation_funcs = [Sigmoid(), Sigmoid(), Identity()]

net = NeuralNetwork(network_input_size,
        layer_output_sizes,
        activation_funcs,
        MSE(),
    )

print(net.cost_batch(x_train,y_train - y_mean))

net.train(GradientDescent(step_method=RMSpropStep(learning_rate=0.016, decay_rate=0.9),inputs = x_train, targets = y_train - y_mean),100)

print(net.cost_batch(x_train,y_train - y_mean))


Testing MSE for OLS:  0.00020994175283362937
(400,)
